In [1]:
!pip freeze > requirements.txt

In [4]:
from google.colab import files
files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
!python --version

Python 3.8.16


In [ ]:
!pip install pingouin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 16.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import scipy.stats as stats
import numpy as np
from itertools import product

In [ ]:
# mAP scores (Mean Average Precision) for 10 test datasets

model_a = np.array([0.71, 0.74, 0.79, 0.75, 0.76, 0.75, 0.76, 0.72, 0.77, 0.77])
model_b = np.array([0.78, 0.81, 0.86, 0.83, 0.84, 0.83, 0.83, 0.79, 0.85, 0.84])

In [ ]:
# Cliff's Delta Function
def cliffs_delta(lst1, lst2):
    n = len(lst1)
    m = len(lst2)
    more = sum(x > y for x, y in product(lst1, lst2))
    less = sum(x < y for x, y in product(lst1, lst2))
    delta = (more - less) / (n * m)

    # Interpretation (Romano et al. 2006)
    abs_delta = abs(delta)
    if abs_delta < 0.147:
        size = 'negligible'
    elif abs_delta < 0.33:
        size = 'small'
    elif abs_delta < 0.474:
        size = 'medium'
    else:
        size = 'large'
    return delta, size

In [ ]:
# Cohen's d Function
def cohens_d(x, y):
    nx, ny = len(x), len(y)
    pooled_std = np.sqrt(((nx - 1) * np.std(x, ddof=1) ** 2 + (ny - 1) * np.std(y, ddof=1) ** 2) / (nx + ny - 2))
    d = (np.mean(x) - np.mean(y)) / pooled_std
    return d

In [ ]:
# Compute performance difference
diff = model_b - model_a

# Step 1: Check normality of differences
shapiro_test = stats.shapiro(diff)
p_shapiro = shapiro_test.pvalue

print(f"Shapiro-Wilk test for normality: p = {p_shapiro:.4f}")

Shapiro-Wilk test for normality: p = 0.0002


In [ ]:
# Step 2: Choose the appropriate test
if p_shapiro > 0.05:
    # Differences are normally distributed -> Use paired t-test
    t_stat, p_ttest = stats.ttest_rel(model_a, model_b)
    print(f"Paired t-test: t = {t_stat:.4f}, p = {p_ttest:.4f}")
    if p_ttest < 0.05:
        print("🔹 Significant improvement in performance (p < 0.05).")
    else:
        print("⚠️ No significant difference (p ≥ 0.05).")
else:
    # Differences are not normally distributed -> Use Wilcoxon test
    wilcoxon_stat, p_wilcoxon = stats.wilcoxon(model_a, model_b)
    print(f"Wilcoxon test: W = {wilcoxon_stat}, p = {p_wilcoxon:.4f}")
    if p_wilcoxon < 0.05:
        print("🔹 Significant improvement in performance (p < 0.05).")
    else:
        print("⚠️ No significant difference (p ≥ 0.05).")

Wilcoxon test: W = 0.0, p = 0.0020
🔹 Significant improvement in performance (p < 0.05).


In [ ]:
# Apply Cliffs_delta
delta, delta_size = cliffs_delta(model_a, model_b)
delta = abs(delta)
print(f"Cliff’s δ: {delta:.3f} ({delta_size} effect)")

Cliff’s δ: 0.970 (large effect)


In [ ]:
# Cohen
d = cohens_d(model_a, model_b)
print("=== Effect Size Results ===")
d = abs(d)
print(f"Cohen’s d: {d:.3f} ", end='')

if abs(d) < 0.2:
    print("(negligible)")
elif abs(d) < 0.5:
    print("(small)")
elif abs(d) < 0.8:
    print("(medium)")
else:
    print("(large effect)")

=== Effect Size Results ===
Cohen’s d: 2.993 (large effect)
